In [2]:
import os
import torch
from layers import SpatialTransformer
import numpy as np
import nibabel as nib

from skimage.transform import resize
from scipy.ndimage.interpolation import affine_transform

from deformation_simulator import Simulator, SourceGen
from helpers import normalizeImage, random_transform, compute_negative_jacobian
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

In [8]:
folder = "/home/myi16/jsr_Y/new_flow"
write_dir = "/home/myi16/jsr_Y/new_val"

In [3]:
import random
index_list = random.choices(range(0,400),k=10)
index_list

[376, 167, 222, 394, 331, 337, 188, 73, 197, 147]

In [23]:
def mynorm(img,maxi,mini):
    return (maxi-mini)*(img-img.min())/(img.max()-img.min())+mini

In [28]:
def dataread(data_dict):
    
    fixed_img = data_dict['fixed_img']
    moving_img = data_dict['moving_img']
    fixed_warp = data_dict['fixed_warp']
    moving_truth = data_dict['moving_truth']
    ventricle_fixed = data_dict['ventricle_fixed']
    ventricle_moving = data_dict['ventricle_moving']
    affine = data_dict['affine']
    flow = data_dict['flow']
    cbct = data_dict['cbct']
    cbct_warp = data_dict['cbct_warp']

    maxi = moving_truth.max()
    mini = moving_truth.min()

    moving_img = mynorm(moving_img,maxi,mini)
    output_dict = {}
    output_dict['ct_fixed'] = fixed_img 
    output_dict['mr_moving'] = torch.from_numpy(moving_img).unsqueeze(0).unsqueeze(0)
    output_dict['ct_moving'] = fixed_warp 
    output_dict['moving_truth'] = moving_truth
    output_dict['seg_fixed'] = torch.from_numpy(ventricle_fixed).unsqueeze(0).unsqueeze(0) 
    output_dict['seg_moving'] = torch.from_numpy(ventricle_moving).unsqueeze(0).unsqueeze(0) 
    output_dict['affine'] = affine 
    output_dict['flow'] = flow 
    output_dict['cbct_fixed'] = torch.from_numpy(cbct).unsqueeze(0).unsqueeze(0)
    output_dict['cbct_warp'] = cbct_warp 
    return output_dict

In [ ]:
for key in dataread()

In [29]:
for i in range(10):
    name = f'/{index_list[i]:03}.npz'
    data = np.load(folder+name)
    output = dataread(data)
    torch.save(output,write_dir+f'/{i:03}.pt')

In [9]:
### Test
data = torch.load(write_dir+f'/{0:03}.pt')
type(data)


dict

In [10]:
for key in data.keys():
    print(data[key].max())

1.0
tensor(0.6334)
1.0
0.6333673
tensor(138.0000)
tensor(138.0000)
4.325005053620965
2.9506998
tensor(1.)
1.0


In [48]:
data = torch.load('/mnt/blackhole-data2/RadOnc_Brain/Validation'+'/valid_0.pt')

In [11]:
for keys in data.keys():
    print(keys)
    print(data[keys].shape)
    print(type(data[keys]))

ct_fixed
(128, 160, 128)
<class 'numpy.ndarray'>
mr_moving
torch.Size([1, 1, 128, 160, 128])
<class 'torch.Tensor'>
ct_moving
(128, 160, 128)
<class 'numpy.ndarray'>
moving_truth
(128, 160, 128)
<class 'numpy.ndarray'>
seg_fixed
torch.Size([1, 1, 128, 160, 128])
<class 'torch.Tensor'>
seg_moving
torch.Size([1, 1, 128, 160, 128])
<class 'torch.Tensor'>
affine
(4, 4)
<class 'numpy.ndarray'>
flow
(3, 128, 160, 128)
<class 'numpy.ndarray'>
cbct_fixed
torch.Size([1, 1, 128, 160, 128])
<class 'torch.Tensor'>
cbct_warp
(128, 160, 128)
<class 'numpy.ndarray'>
